# Sorting out basic design issues

## Fixing the num_rounds -- was missing from earlier

In [44]:
import numpy as np
import random
from collections import Counter

class User:
    """
    This is a relatively simple class that models the user (i,e the one that clicks the ad linkk or not)
    The primary function is the user response to when an ad is shown (ie show_ad)
    """
    # import numpy as np
    # import random
    # from collections import Counter
    
    def __init__(self):
        # Initializes the probability with the random-uniform distribution
        self.__probability = random.uniform(0, 1)

    def __repr__(self):
        return "this is a user with a secret click probability of " + str(self.__probability)
    
    def __str__(self):
        return self.__repr__()
    
    
    def show_ad(self):
        # The show ad function is simple in that it returns a True / False value based on 
        # a random choice of the probability distribution
        return  np.random.choice((True, False), p = [self.__probability, 1-self.__probability])

class Auction:
    
    """
    The auction class sets the bidding rules and manages both the bidders and users
    It also determines the winning bid and the price (which is the second highest)
    Also, included is the interaction with the bidder to request and then notify the bidder of the outcome
    The user function models the user (i,e the one that clicks the ad linkk or not)
    
    Key Assumption: Given the max reward is $1, we will limit max bids to at most $5
    """
    
    # import numpy as np
    # import random
    # from collections import Counter
    
    
    def __init__(self, users, bidders):
        
        # Initializes the auction class with users, bidders, auction history
        # balances are tracked by bidder
        
        self.users = users
        self.bidders = bidders
        self.auction_history = []
        
        # To note, I have used id(bidder) instead of (bidder) as that is incomprehensible
        self.balances = {id(bidder): 0 for bidder in bidders} ########### NEW LINE
        
        #print("*"*100, '\n')
        #print('Beginning balances : ',self.balances, '\n','-'*100)
        
        #self.num_rounds = 0
        self.auction_active_bidder = True
        
    def __repr__(self):
        return "This is a second price auction with " + str(len(self.users)) + " users and " + str(len(self.bidders)) + " bidders"
    
    def __str__(self):
        return self.__repr__()    
    
    def execute_round(self):
        
        # while self.auction_active_bidder == True:
        
        ############  EXECUTE ROUND  #### This is the principal function orchestrator #####################
        ###################################################################################################

        ############ NUMBER OF AUCTION ROUNDS -- Passed from Bidder #######################################

#         for bidder in self.bidders:

#             self.num_rounds = bidder.num_rounds

        all_invalid_bidders = any(x > -1000 for x in self.balances.values())


        #########  if the -ve balance of ALL bidders is < -1000, then break ###############################

        while not all_invalid_bidders:
            return "no bidders are qualifed as all balances are below -1000"
            break

        #########  Proceed with the auction is even 1 bidder has a balance > -1000 #########################
        ####################################################################################################

        else:
            # Initialize dictionary and lists for tracking bids
            bids_dict = {}
            bids_list = []
            bids_list_raw = []

            ########## STEP 1. SELECT USER (from the pool of users) ########################################

            chosen_user = random.randint(0, len(self.users)-1)
            #print('Selected User: ', id(chosen_user), '\n','-'*100) 

            ########### STEP 2. COLLECT BIDS (send to all bidders) #########################################

            for bidder in self.bidders:
                if self.balances[id(bidder)] > -1000:
                    bids_dict[bidder] = bidder.bid(chosen_user)
                    
                    if bidder.bid(chosen_user) == 0:
                        print('this is a zero bid need to exit loop', '\n','-'*100)
                    #self.bidding_round = bidder.bidding_round

                    self.auction_history.append((id(bids_dict[bidder]), bids_dict[bidder])) #bidder
                    highest_bid = 0
                    winning_price = 0
                else:
                    print("Bidder ", id(bidder), "balance is less than -1000. Cannot continue bid" )
     #               #raise Exception("Bidder's balance is less than -1000. Cannot continue bid")
                    continue

            #print("Current bidding Round: ", self.bidding_round, '\n','-'*100)

            ############ STEP 3. DETERMINE WINNING BID ####################################################

            for bidder, bid_value in bids_dict.items():

                bids_list_raw.append((bidder, bid_value))
                ## only needed to track a human readable bidder id
                bids_list.append((id(bidder), bid_value))
                
                print("bids list", bids_list, '\n','-'*100)
                

            ########### STEP 4. SORT THE Bids List based on price #########################################

            sorted_list = sorted(bids_list, key=lambda t:t[1])    
            
            print('Sorted bids list', sorted_list, '\n','-'*100 )

            #------------- for analysis only - creating a copy with id(bidder) ---------------------------
            sorted_list_raw = sorted(bids_list_raw, key=lambda t:t[1])
            print('sorted bids list RAW', sorted_list_raw, '\n','-'*100 )


            ########### STEP 5. Select second highest price ###############################################

            if len(sorted_list) > 1:
                winning_price = (sorted_list_raw)[-2][1]
                winning_bidder = (sorted_list_raw)[-1][0]
            else:
                winning_price = (sorted_list_raw)[0][-1]
                winning_bidder = (sorted_list_raw)[0][0]

    #        print('Winning Price: ', winning_price, '\n','-'*100)
    #        print('winning bidder', winning_bidder, '\n','-'*100)

            ########### STEP 6. Determine if there are more than 1 bidders with same price ################

            multiple_winning_bidders = [tup for tup in sorted_list_raw if tup[1] == winning_price]

            if len(multiple_winning_bidders)>1:
                #print('List of biders that submitted similar bids: ' , multiple_winning_bidders, '\n','-'*100)
                randindex = random.randint(0, len(multiple_winning_bidders)-1)
                winning_bidder = multiple_winning_bidders[randindex][0]
                #print('Winning bidder from list of similar priced bids: ', winning_bidder, '\n','-'*100)
            else: 
                pass
                #winning_bidder = multiple_winning_bidders[0][0]

            #print('Winning Bidder: ', id(winning_bidder), '\n','-'*100)
            #print('Winning Price: ', winning_price, '\n','-'*100)

            ############ STEP 7. Validate USER CLICK after SHOW AD ###########################################

            ad_result = self.users[chosen_user].show_ad()
            #print('User Clicked on Ad?: ', ad_result,'\n','-'*100)

            ############# STEP 8.NOTIFY BIDDER & UPDATE BALANCES #############################################

            for bidder in self.bidders:
                if bidder == winning_bidder and ad_result == True:
                    #print(id(bidder), "bidder == winning_bidder and ad_result == True:")
                    self.balances[id(bidder)] -= winning_price
                    self.balances[id(bidder)] += 1

                    bidder.notify(True, winning_price, ad_result)

                elif bidder == winning_bidder and ad_result == False:
                    #print(id(bidder), "bidder == winning_bidder and ad_result == False")
                    self.balances[id(bidder)] -= winning_price

                    bidder.notify(True, winning_price, ad_result)

                else:
                    #print(id(bidder), "Did not win bid - only notify")
                    bidder.notify(False, winning_price, None)
            #print('-'*100, '\n','balances at end of bidding round', self.balances,'\n','-'*100)

#                 ############# STEP 9.EXIT AUCTION ##### based on num_rounds #######################################

#                 self.bidding_round += 1

#                 if self.bidding_round >  self.num_rounds:
#                     #print("End of bidding round")
#                     self.auction_active_bidder = False
#                     break
        
class Bidder:
    
    """
    This is the bidder class where we track bidding details, patterns and history. 
    In addition to the required functions, there are five other functions that have been 
    created to model bidding strategies -- listed below
    
    Note the acronyms user
    BWCY: Bidder Won, Clicked Yes
    BWCN: Bidder Won, Clicked No
    BLOS: Bidder Lost
    """
    # import numpy as np
    # import random
    # from collections import Counter
    
    def __init__(self, num_users, num_rounds):
            
    ############ Initialize Attributes for bid and history  ###########################################
    ###################################################################################################
        
        ########### initializing basic attributes ###########
        self.num_users = num_users
        self.num_rounds = num_rounds
        self.bidding_round = 0
        self.bid_price = 0
        self.active_bidder = True

        ########### Basic tracking metrics - dictionaries ###########
        self.bid_participate = {i: 0 for i in range(num_users)} #whether bidder bids or not (USER: BID COUNT)
        self.bid_wins_no_click = {i: 0 for i in range(num_users)} # if bidder wins
        self.bid_win_user_clicks = {i: 0 for i in range(num_users)} # if user clicks
        self.bid_lost = {i: 0 for i in range(num_users)}

        ########### Historical transaction tracking for trend analysis ###########
        self.bid_win_click_no = []
        self.bid_win_click_yes = []
        self.bid_lost_list = []
        self.bid_360 = []
        
    ############ SUBMIT BID  ########### initiated by auction #########################################
    ###################################################################################################
    
    def __repr__(self):
        return "This is a bidder participating in a second price auction with " + str(self.num_rounds) + " rounds of bidding"     
    def __str__(self):
        return self.__repr__()   
    
    
    def bid(self, user_id):
        """
        Function to set teh default (aka base) bid price
        The bid price is called from a separate function that determines the bid strategy
        """

        self.bidding_round += 1
    
    ############  STOP BIDDING AFTER NUM_ROUNDS  ########### initiated by auction #####################
    ###################################################################################################
        
        if self.bidding_round > self.num_rounds:
            print("examine here ",'bidding round', self.bidding_round, 'number of rounds', self.num_rounds )
            self.bid_price = 0
            return self.bid_price
            
            #print("Cannot participate in more than these rounds")
        else:    
            self.user_id = user_id
            self.bid_price = self.bid_strategy(user_id)
            return self.bid_price

    ############  NOTIFIED OF BID OUTCOME  ########### initiated by auction ###########################
    ###################################################################################################
    
    def notify(self, auction_winner, price, clicked):
        bid_y_n_won = 0
        self.auction_winner = auction_winner
        self.price = price
        self.clicked = clicked
        
        if auction_winner == True and clicked == True:
            self.bid_win_user_clicks[self.user_id] = (self.bid_win_user_clicks).get(self.user_id, 0)+1
            self.bid_win_click_yes.append(['BWCY', self.bidding_round, id(self), self.user_id, self.auction_winner,self.bid_price, self.price, self.clicked])
            self.bid_360.append(['BWCY', self.bidding_round, id(self), self.user_id, self.auction_winner,self.bid_price, self.price, self.clicked])

        elif auction_winner == True and clicked == False:
            self.bid_wins_no_click[self.user_id] = (self.bid_wins_no_click).get(self.user_id, 0)+1
            self.bid_win_click_no.append(['BWCN', self.bidding_round, id(self), self.user_id, self.auction_winner,self.bid_price, self.price, self.clicked])
            self.bid_360.append(['BWCN', self.bidding_round, id(self), self.user_id, self.auction_winner,self.bid_price, self.price, self.clicked])
        
        else:
            self.bid_lost[self.user_id] = (self.bid_lost).get(self.user_id, 0)+1
            self.bid_lost_list.append(['BLOS',self.bidding_round, id(self), self.user_id, self.auction_winner,self.bid_price, self.price, self.clicked])
            self.bid_360.append(['BLOS',self.bidding_round, id(self), self.user_id, self.auction_winner,self.bid_price, self.price, self.clicked])
    
    ############  BID STRATEGY - CONTROL FLOW  ########################################################
    ###################################################################################################
    
    def bid_strategy(self, user_id):

        """
        This function controls the strategy and changes this with what it learns from the bidding process
        There are two basic - default strategies that will be followed for the first 50 rounds which are
        (1) start slow and observe bidding behavior for first 10 rounds
        (2) go aggressive and match max price of past rounds and in crease base bid by 10%
        After the first 50 rounds, the strategy revolves through 4 models (A, B, C & D) aso detailed below
        """
        
        ########## SET THE BASE BID PRICE  ##### Revise this based on Strategy ############################
        #################### Ranges from 0 to $1 ##########################################################
        
        self.base_bid_price = random.randint(0, 10000)/10000 #random.uniform(0, 1)
        
        ###################################################################################################
        
        trx_master = [x for x in self.bid_360]
        strategy_list = ['A', 'B', 'C', 'D']
        strategy = random.choice(strategy_list)
        
        for item in trx_master:
        
        ########## Observe: Start Slow for the first 10 rounds  ###########################################
        ###################################################################################################
        
            if item[1] < 10:
                self.bid_price = 0.01
            
        ########## Next 40 rounds, match max price + 10%  #################################################
        ###################################################################################################
            
            elif item[1] >= 10 and item[1] <= 50 :
                self.bid_price = max(self.bid_360, key=lambda x: x[6])[6]*1.10
                if self.bid_price > self.base_bid_price * 100:
                    self.bid_price = self.base_bid_price
        
        ########## Rounds 50+ Learn, Optimize, Mitigate Losses and Win  ###################################
        ###################################################################################################
        
            elif strategy == 'A':
                self.bid_price = self.strategy_a()
            
            elif strategy == 'B':
                self.bid_price = self.strategy_b()
                
            elif strategy == 'C':
                self.bid_price = self.strategy_c()
                
            elif strategy == 'D':
                self.bid_price = self.strategy_d()
            
            else:
                self.bid_price = self.base_bid_price
            
        
        return self.bid_price
        
    
    ############  STRATEGY A - Identify losses and counter  ###########################################
    ###################################################################################################
        
    def strategy_a(self):
            
        """
        Setting min bid above average difference of loss for prior 10 bids  was greater than 10%
        then adjust you next bid to in increase by that amount
        """

        # filter last 10 losses  
        a = [x for x in self.bid_360[-10:] if x[0] == 'BLOS' and (x[6]-x[5]) > (.1*x[5])]

        # calculated the average percentage loss (> 10%)
        avg_loss_10percent = ((sum(i[6] for i in a) - sum(i[5] for i in a))/(max(1, len(a))))

        #adjust bid price
        self.bid_price = self.base_bid_price + avg_loss_10percent
        if self.bid_price > self.base_bid_price * 2:
            self.bid_price = self.base_bid_price
        return self.bid_price

    ############  STRATEGY B - Exploit/Focus on top User   ############################################
    ###################################################################################################
    
    def strategy_b(self):
        #self.user_id = user_id
        
        """
        Focus on most successful user --- ie where bidder has worn atleast 5 times for same user
        For this user, double the base bid price to increase odds of winning again
        """
        
        a = [x for x in self.bid_360[-100:] if x[0] == 'BWCY']
        #print(a)
        b = (Counter([x[3] for x in a]))
        #print(b)
        # filter for success > 5 times
        c = { k: v for k, v in b.items() if v >= 0 }
        # get max user (most successful thus far)
        d = max(c, key=c.get, default=0) # get user with max wins
        
        if d == None:
            self.bid_price = self.base_bid_price
        elif self.user_id ==  d:
            self.bid_price = self.base_bid_price*2
        else:    
            self.bid_price = self.base_bid_price
        return self.bid_price
        
    
    ############  STRATEGY C - Identify Max leaders and counter   #####################################
    ###################################################################################################
    
    def strategy_c(self):
        
        """
        Find Max winning bid in past 10 bids and increase your base bid by 20%
        """

        a = [x for x in self.bid_360[-10:] if x[0] == 'BLOS']
        
        if a == []:
        #if max(a, key=lambda x: x[6]) == []:
        # if max(a, key=lambda x: x[6])[6] == False:
            self.bid_price = self.base_bid_price
        else:
            self.bid_price = max(a, key=lambda x: x[6])[6]*1.20
            if self.bid_price > self.base_bid_price * 5:
                self.bid_price = self.base_bid_price
        return self.bid_price
    
    
    ############  STRATEGY D - Conserve cash when <-500   #############################################
    ###################################################################################################
    
    def strategy_d(self):
            
        """
        Monitor if balance goes < -500 then scale back on the bids by 10% of the base price
        Conserve cash and wait for other random strategies A, B or C to try again
        """

        # sum all bids that were won, but user not clicked (ie BWCN)
        a = [x for x in self.bid_360 if x[0] == 'BWCN'] 

        # calculate cost of bids in BWCN
        b = (sum(i[6] for i in a))
        
        # sum of all bids where user clicked (ie BWCY)
        c = [x for x in self.bid_360 if x[0] == 'BWCY']
        # sum of total cost
        d = (sum(i[6] for i in c) )

        # earnings till date (ie $1 for every BWCY)
        e = len(c)
        f = e * 1

        current_balance = (b + d -f)
        
        if current_balance < -500:
            self.bid_price = 0.01
        return self.base_bid_price

# b1, b2, b3, b4, b5, b6, b7, b8, b9, b10 = Bidder(1,5), Bidder(1,7), Bidder(1,8), Bidder(1,10), Bidder(1,10), Bidder(1,10), Bidder(1,10), Bidder(1,10), Bidder(1,10), Bidder(1,10)
# auction = Auction( [User()for i in range(10)], [b1, b2, b3, b4, b5, b6, b7, b8, b9, b10])
# auction.execute_round()
    

In [47]:
#b1, b2, b3 = Bidder(1,10), Bidder(1,10), Bidder(1,10)
b1, b2, b3, b4, b5, b6, b7, b8, b9, b10 = Bidder(1,10), Bidder(1,10), Bidder(1,10), Bidder(1,10), Bidder(1,10), Bidder(1,10), Bidder(1,10), Bidder(1,10), Bidder(1,10), Bidder(1,10)
auction = Auction( [User()for i in range(10)], [b1, b2, b3, b4, b5, b6, b7, b8, b9, b10])
#auction = Auction( [User(), User()], [b1, b2, b3])
auction.execute_round()

this is a zero bid need to exit loop 
 ----------------------------------------------------------------------------------------------------
this is a zero bid need to exit loop 
 ----------------------------------------------------------------------------------------------------
this is a zero bid need to exit loop 
 ----------------------------------------------------------------------------------------------------
this is a zero bid need to exit loop 
 ----------------------------------------------------------------------------------------------------
this is a zero bid need to exit loop 
 ----------------------------------------------------------------------------------------------------
this is a zero bid need to exit loop 
 ----------------------------------------------------------------------------------------------------
this is a zero bid need to exit loop 
 ----------------------------------------------------------------------------------------------------
this is a zero bid n

In [41]:
auction.execute_round()

bids list [(140682061741456, 0.01)] 
 ----------------------------------------------------------------------------------------------------
bids list [(140682061741456, 0.01), (140682061740736, 0.01)] 
 ----------------------------------------------------------------------------------------------------
bids list [(140682061741456, 0.01), (140682061740736, 0.01), (140682061741168, 0.01)] 
 ----------------------------------------------------------------------------------------------------
bids list [(140682061741456, 0.01), (140682061740736, 0.01), (140682061741168, 0.01), (140682061741552, 0.01)] 
 ----------------------------------------------------------------------------------------------------
bids list [(140682061741456, 0.01), (140682061740736, 0.01), (140682061741168, 0.01), (140682061741552, 0.01), (140682061740544, 0.01)] 
 ----------------------------------------------------------------------------------------------------
bids list [(140682061741456, 0.01), (140682061740736, 0

In [42]:
[auction.execute_round() for i in range(1)]

bids list [(140682061741456, 0.01)] 
 ----------------------------------------------------------------------------------------------------
bids list [(140682061741456, 0.01), (140682061740736, 0.01)] 
 ----------------------------------------------------------------------------------------------------
bids list [(140682061741456, 0.01), (140682061740736, 0.01), (140682061741168, 0.01)] 
 ----------------------------------------------------------------------------------------------------
bids list [(140682061741456, 0.01), (140682061740736, 0.01), (140682061741168, 0.01), (140682061741552, 0.01)] 
 ----------------------------------------------------------------------------------------------------
bids list [(140682061741456, 0.01), (140682061740736, 0.01), (140682061741168, 0.01), (140682061741552, 0.01), (140682061740544, 0.01)] 
 ----------------------------------------------------------------------------------------------------
bids list [(140682061741456, 0.01), (140682061740736, 0

[None]

In [48]:
[auction.execute_round() for i in range(10)]

bids list [(140682061704400, 0.01)] 
 ----------------------------------------------------------------------------------------------------
bids list [(140682061704400, 0.01), (140682061704640, 0.01)] 
 ----------------------------------------------------------------------------------------------------
bids list [(140682061704400, 0.01), (140682061704640, 0.01), (140682061704928, 0.01)] 
 ----------------------------------------------------------------------------------------------------
bids list [(140682061704400, 0.01), (140682061704640, 0.01), (140682061704928, 0.01), (140682061704448, 0.01)] 
 ----------------------------------------------------------------------------------------------------
bids list [(140682061704400, 0.01), (140682061704640, 0.01), (140682061704928, 0.01), (140682061704448, 0.01), (140682061704496, 0.01)] 
 ----------------------------------------------------------------------------------------------------
bids list [(140682061704400, 0.01), (140682061704640, 0

[None, None, None, None, None, None, None, None, None, None]

In [38]:
b1, b2 = Bidder(1,10), Bidder(1,10)
auction = Auction( [User()for i in range(5)], [b1, b2])

#auction = Auction( [User(), User()], [b1, b2, b3])
auction.execute_round()
#[auction.execute_round() for i in range(500)]

In [32]:
b1 = Bidder(1,100)
auction = Auction( [User()for i in range(10)], [b1])

#auction = Auction( [User(), User()], [b1, b2, b3])
auction.execute_round()
#[auction.execute_round() for i in range(500)]

In [38]:
bal = auction.balances
bal

{140682867466448: 0,
 140682867467504: 0,
 140682867466640: -0.01,
 140682867466352: 0,
 140682867467888: 0,
 140682867466496: 1,
 140682867467600: 0.97,
 140682867469184: 0.98,
 140682867469568: 0.98,
 140682867467792: -0.01}

In [28]:
[auction.execute_round() for i in range(100)]

Cannot participate in more than these rounds
Cannot participate in more than these rounds
Cannot participate in more than these rounds
Cannot participate in more than these rounds
Cannot participate in more than these rounds
Cannot participate in more than these rounds
Cannot participate in more than these rounds
Cannot participate in more than these rounds
Cannot participate in more than these rounds
Cannot participate in more than these rounds


TypeError: '<' not supported between instances of 'NoneType' and 'NoneType'

In [ ]:
b1

# Consolidate Metrics: All Bidders

## Overall

In [ ]:
# Summary Counts
print('Bidding Rounds', b1.bidding_round)

BWCY = (len(b1.bid_win_click_yes), len(b2.bid_win_click_yes), \
      len(b3.bid_win_click_yes), len(b4.bid_win_click_yes),  len(b5.bid_win_click_yes), len(b6.bid_win_click_yes), \
      len(b7.bid_win_click_yes), len(b8.bid_win_click_yes), len(b9.bid_win_click_yes), len(b10.bid_win_click_yes))

sum(BWCY)
print('BWCY - Bid Win - Click YES: Count', sum(BWCY))

BWCN = (len(b1.bid_win_click_no),len(b2.bid_win_click_no),len(b3.bid_win_click_no)\
     ,len(b4.bid_win_click_no),len(b5.bid_win_click_no),len(b6.bid_win_click_no),len(b7.bid_win_click_no),\
    len(b8.bid_win_click_no),len(b9.bid_win_click_no),len(b10.bid_win_click_no))

sum(BWCN)

print('BWCN - Bid Win - Click NO: Count',sum(BWCN))

# GENERAL CLICK RATIO

print('Overall Click Ratio ', sum(BWCY) / (sum(BWCY)+sum(BWCN)) )

# BLOS = NOT RELEVANT
# print('BLOS - Bid LOST: Count',len(b1.bid_lost_list))

# print('Balances', auction.balances)

# Bid Refining - Filters

## Observe first 10 rounds

In [ ]:
trx_master = [x for x in self.bid_360]

for item in trx_master:

    if item[1] < 10:
        self.bid_price = 0.01
    else:
        self.bid_price = base_bid_price

# trx_master = [x for x in b1.bid_360]

# for item in trx_master:
#     if item[1] < 10:
#         print('hey')
#     else:
#         print('hey no')

## Loss by > 10%

In [ ]:
# lost the bid by a price > 10%

# filter last 10 bids for those you have lost 
a = [x for x in b1.bid_360[-10:] if x[0] == 'BLOS' and (x[6]-x[5]) > (.1*x[5])]
#print(a)

# calculated the average percentage loss (> 10%)
b = ((sum(i[6] for i in a) - sum(i[5] for i in a))/(max(1, len(a))))

print(b)

# then set bid_price = 

# [0]Category, [1]self.bidding_round, [2]id(self), [3]self.user_id, 
# [4]self.auction_winner, [5]#self.bid_price, [6]self.winning price, [7]self.clicked

## Most successful user

In [ ]:
# lost the bid by a price > 10%
# user_id = 9
from collections import Counter

a = [x for x in b1.bid_360[-1000:] if x[0] == 'BWCY']
#print(a)
b = (Counter([x[3] for x in a]))
#print(b)
c = { k: v for k, v in b.items() if v >= 0 }
print(c)
d = max(c, key=c.get, default = 0) # get user with max wins
print(d)
#print(c)

# if user_id == max(c, key=c.get, default = 0):
#     print(user_id)
#print(max(b, key=b.get))

# then set bid_price = 

# [0]Category, [1]self.bidding_round, [2]id(self), [3]self.user_id, 
# [4]self.auction_winner, [5]#self.bid_price, [6]self.winning price, [7]self.clicked

## Wait certain rounds, then accelerate

In [ ]:
#a = [x for x in b1.bid_360[-100:] if x[0] == 'BWCN'] # < ]
a = [x for x in b1.bid_360[-100:] if 100 < x[1] < 200] #'BWCN'] # < ]
print(a)
     # then take default bid probability

## Match Max and go over 10%

---

In [ ]:
b1.bid_360 # Category,self.bidding_round, id(self), self.user_id, self.auction_winner,
           #self.bid_price, self.winning price, self.clicked
#[x for x in b1.bid_360]    

a = [x for x in b1.bid_360[-10:] if x[0] == 'BLOS']
a
# a  = [x for x in b1.bid_360]
# a = [trx_master[-10:] if x[0] == 'BLOS']
#print(trx_master)
a

b = max(a, key=lambda x: x[6])
b

# b = max(a, key=lambda x: x[6])[6]*1.20
# print(b)

## Monitor balance and go convervative

In [ ]:
# Monitor if balance goes < -500
# then scale back on the bids by 10% of the base price

# filter last 10 bids for those you have lost 
a = [x for x in b1.bid_360 if x[0] == 'BWCN'] 
#print(a)

# calculate current balance
b = (sum(i[6] for i in a))# - sum(i[5] for i in a))/(max(1, len(a))))
print(b)

c = [x for x in b1.bid_360 if x[0] == 'BWCY']
d = (sum(i[6] for i in c) )
print(d)
      
e = len(c)

f = e * 1


balance = (b + d -f)
print(balance)
# then set bid_price = 

---

## By Bidder

### for b1

In [12]:
# Summary Counts
print('Bidding Rounds', b1.bidding_round)
print('BWCY - Bid Win - Click YES: Count', len(b1.bid_win_click_yes))
print('BWCN - Bid Win - Click NO: Count',len(b1.bid_win_click_no))
print('BLOS - Bid LOST: Count',len(b1.bid_lost_list))
print('Balances', auction.balances)

Bidding Rounds 511
BWCY - Bid Win - Click YES: Count 19
BWCN - Bid Win - Click NO: Count 35
BLOS - Bid LOST: Count 457
Balances {140683410538752: 18.990000000000002, 140683410538944: 12, 140682331457280: 17.990000000000002, 140682331457568: 27, 140682331457232: 17.990000000000002, 140682331457328: 11.99, 140682331457184: 26.97, 140682331457664: 19, 140682331457472: 24, 140682331457376: 16.98}


## for b2

In [ ]:
# Summary Counts
print('Bidding Rounds', b2.bidding_round)
print('BWCY - Bid Win - Click YES: Count', len(b2.bid_win_click_yes))
print('BWCN - Bid Win - Click NO: Count',len(b2.bid_win_click_no))
print('BLOS - Bid LOST: Count',len(b2.bid_lost_list))
print('Balances', auction.balances)

## for b3

In [ ]:
# Summary Counts
print('Bidding Rounds', b3.bidding_round)
print('BWCY - Bid Win - Click YES: Count', len(b3.bid_win_click_yes))
print('BWCN - Bid Win - Click NO: Count',len(b3.bid_win_click_no))
print('BLOS - Bid LOST: Count',len(b3.bid_lost_list))
print('Balances', auction.balances)

---

# User Info

In [ ]:
print('Bidding Rounds', b1.bidding_round)
print('BWCY - Bid Wins User Clicked', {k: v for k, v in sorted(b1.bid_win_user_clicks.items(), key=lambda item: item[0])})
print('BWCN - Bid Wins User NO Clicked', {k: v for k, v in sorted(b1.bid_wins_no_click.items(), key=lambda item: item[0])})
print('BLOS - Bids Lost', {k: v for k, v in sorted(b1.bid_lost.items(), key=lambda item: item[0])})


In [ ]:
b1.bid_360